# Machine Learning for NLP: Project 2

Lucas Artaud & Iswarya Sivasubramaniam DIA 1

In [41]:
import os
import pandas as pd

## Data loading

In [46]:
folder = 'Traduction avis clients'

df = pd.DataFrame()

for i in range(1, 36):
    filename = f'avis_{i}_traduit.xlsx'
    path = os.path.join(folder, filename)
    df = pd.concat([df, pd.read_excel(path)], ignore_index=True)

df

,note,auteur,avis,assureur,produit,type,date_publication,date_exp,avis_en,avis_cor,avis_cor_en
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,train,06/09/2021,01/09/2021,"Best insurance, price, solutions, listening, s...",NaN,NaN
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,train,03/05/2021,01/05/2021,"I am generally satisfied, except that you have...",NaN,NaN
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,train,21/03/2021,01/03/2021,Very affordable price Several options are avai...,NaN,NaN
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,train,10/06/2021,01/06/2021,"I satisfy the service, a very fast response fr...",NaN,NaN
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,train,29/01/2017,01/01/2017,"Customer for more than 25 years, very disappoi...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
34430,NaN,mamybouly-129360,"Bonjour, j'ai une mauvaise expérience avec la ...",Sma,garantie-decennale,test,24/08/2021,01/08/2021,"Hello, I have a bad experience with the Socoba...","Bonjour, j'ai une mauvaise expérience avec la ...","Hello, I have a bad experience with the Socoba..."
34431,NaN,tarti-88637,Entreprise très impactée par le covid- secteur...,Allianz,assurances-professionnelles,test,18/05/2021,01/05/2021,Company very impacted by the covid- event sect...,Entreprise très impactée par le covid- secteur...,NaN
34432,NaN,victor-109599,"Suite a un cambriolage le 22/01/2021, je suis ...",Allianz,assurances-professionnelles,test,08/04/2021,01/04/2021,"Following a burglary on 01/22/2021, I am still...","Suite a un cambriolage le 22/01/2021, je suis ...",NaN
34433,NaN,lkio-110907,"Bonjour,\n\nJe constate qu'entre le moment de ...",Generali,assurances-professionnelles,test,19/04/2021,01/04/2021,"Hello,\n\nI note that between the time of my r...","Bonjour,\n\nJe constate qu'entre le moment de ...",NaN


## Data cleaning

In [47]:
df.drop(columns=['type', 'avis_en', 'avis_cor', 'avis_cor_en'], inplace=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017
...,...,...,...,...,...,...,...
34430,NaN,mamybouly-129360,"Bonjour, j'ai une mauvaise expérience avec la ...",Sma,garantie-decennale,24/08/2021,01/08/2021
34431,NaN,tarti-88637,Entreprise très impactée par le covid- secteur...,Allianz,assurances-professionnelles,18/05/2021,01/05/2021
34432,NaN,victor-109599,"Suite a un cambriolage le 22/01/2021, je suis ...",Allianz,assurances-professionnelles,08/04/2021,01/04/2021
34433,NaN,lkio-110907,"Bonjour,\n\nJe constate qu'entre le moment de ...",Generali,assurances-professionnelles,19/04/2021,01/04/2021


In [48]:
df.isna().sum()

note                10331
auteur                  1
avis                    0
assureur                0
produit                 0
date_publication        0
date_exp                0
dtype: int64

In [49]:
df.dropna(inplace=True)
df

,note,auteur,avis,assureur,produit,date_publication,date_exp
0,5.0,brahim--k-131532,"Meilleurs assurances, prix, solutions, écoute,...",Direct Assurance,auto,06/09/2021,01/09/2021
1,4.0,bernard-g-112497,"je suis globalement satisfait , sauf que vous ...",Direct Assurance,auto,03/05/2021,01/05/2021
2,5.0,virginie-t-107352,Prix tres abordable plusieurs options s'offren...,Direct Assurance,auto,21/03/2021,01/03/2021
3,4.0,boulain-f-116580,"je satisfait du service, une réponse très rapi...",L'olivier Assurance,auto,10/06/2021,01/06/2021
4,1.0,ouaille31-51798,"Client depuis plus de 25 ans, très déçu de cet...",Matmut,auto,29/01/2017,01/01/2017
...,...,...,...,...,...,...,...
24099,1.0,hophop-107522,Assurance moto chez la mutuel des motards en F...,Mutuelle des Motards,moto,22/03/2021,01/03/2021
24100,1.0,tzl-81680,Même les demandes les plus simples n'aboutisse...,Allianz,habitation,06/12/2019,01/12/2019
24101,1.0,jmr-72500-110395,"En décembre 2019, j'ai souscrit à un contrat C...",Cegema Assurances,sante,14/04/2021,01/04/2021
24102,3.0,cris-77532,Je suis assurer à la gmf depuis plus de 15 ans...,GMF,auto,11/07/2019,01/07/2019
